In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
df = pd.read_csv('/content/pokemon.csv')

In [3]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [4]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

In [5]:
df.head()

,isLegendary,Generation,Type_1,Type_2,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Color,Egg_Group_1,Height_m,Weight_kg,Body_Style
0,False,1,Grass,Poison,45,49,49,65,65,45,Green,Monster,0.71,6.9,quadruped
1,False,1,Grass,Poison,60,62,63,80,80,60,Green,Monster,0.99,13.0,quadruped
2,False,1,Grass,Poison,80,82,83,100,100,80,Green,Monster,2.01,100.0,quadruped
3,False,1,Fire,NaN,39,52,43,60,50,65,Red,Monster,0.61,8.5,bipedal_tailed
4,False,1,Fire,NaN,58,64,58,80,65,80,Red,Monster,1.09,19.0,bipedal_tailed


In [6]:
df['isLegendary'] = df['isLegendary'].astype(int)

In [11]:
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)

In [12]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])

In [14]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

In [15]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [16]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [17]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

In [18]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

In [19]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 1s 2ms/step - loss: 0.6103 - accuracy: 0.7965
Epoch 2/400
18/18 [==============================] - 0s 2ms/step - loss: 0.4590 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3827 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3407 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 2ms/step - loss: 0.3150 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2984 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2866 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2777 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2704 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 2ms/step - loss: 0.2643 - accuracy: 0.9263
Epoch 11/

In [21]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print('Our test accuracy was {}'.format(accuracy_value))

5/5 [==============================] - 0s 3ms/step - loss: 0.0379 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


In [23]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)

In [24]:
predictor(test_data, test_labels, 149)

This was correctly predicted to be a "1"!
